In [2]:
# Data Analysis
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import pickle as pkl
from scipy import sparse

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS

# Text Processing
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning packages
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE

# Model training and evaluation
from sklearn.model_selection import train_test_split

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance

#Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report

# Ignore noise warning
import warnings
warnings.filterwarnings("ignore")

In [100]:
df_wine = pd.read_csv('./wine50_list.csv')

In [101]:
df_wine = df_wine.rename(columns={'Unnamed: 0':'주류'}) # 컬럼명 변경 

In [102]:
df_wine.head()

,주류,중분류,상품명,생산국가,상세정보
0,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,생산자\n세븐 컬러즈 7 Colores\n국가/생산지역\n칠레(Chile) > 센트...
1,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...
2,wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...
3,wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...
4,wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...


In [103]:
df_wine.dtypes

주류      object
중분류     object
상품명     object
생산국가    object
상세정보    object
dtype: object

In [104]:
# 상세정보 전처리 -> 기준으로 삼을 수 있는게 공백밖에 없음 (글자수가 각각 다르기때문)
# 우리가 필요한 정보는 생산자, 주요품종, 음용온도, 알코올, 추천음식 

In [105]:
detail_split = df_wine["상세정보"].str.split("\n")

In [106]:
# wine데이터프레임에 필요한 내용들 추가하기 
df_wine["생산자"] = detail_split.str.get(1)
df_wine["주요품종"] = detail_split.str.get(5)

In [107]:
# 잘 들어온 것을 확인할 수 있음 
df_wine

,주류,중분류,상품명,생산국가,상세정보,생산자,주요품종
0,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,생산자\n세븐 컬러즈 7 Colores\n국가/생산지역\n칠레(Chile) > 센트...,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat..."
1,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)"
2,wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%
3,wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord)
4,wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca..."
5,wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...,기욤 르탕 Guillaume Letang,그르나슈 (Grenache)
6,wine,화이트,"까브 드 루그니, 마콩 루그니 유진 블랑 샤르도네",프랑스,생산자\n까브 드 루그니 Cave de Lugny\n국가/생산지역\n프랑스(Fran...,까브 드 루그니 Cave de Lugny,샤르도네 (Chardonnay) 100%
7,wine,화이트,"깐티네 포베로, 모스카토 다스티 '캄포 델 팔리오'",이탈리아,생산자\n깐띠네 포베로 Cantine Povero\n국가/생산지역\n이탈리아(Ita...,깐띠네 포베로 Cantine Povero,모스카토 (Moscato ) 100%
8,wine,레드,꼬모 까베르네 소비뇽,칠레,생산자\n운두라가 Undurraga\n국가/생산지역\n칠레(Chile) > 센트럴 ...,운두라가 Undurraga,카베르네 소비뇽 (Cabernet Sauvignon)
9,wine,스파클링,노비볼레 로마냐 스푸만테 비앙코,이탈리아,생산자\n까비로 Caviro sca-Forli\n국가/생산지역\n이탈리아(Italy...,까비로 Caviro sca-Forli,"트레비아노 (Trebbiano) , 샤르도네 (Chardonnay) , 파모소 (F..."


In [108]:
detail_split = df_wine["상세정보"].str.split("\n", expand=True) # 데이터프레임형태로 확인하기 

In [109]:
detail_split 

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,생산자,세븐 컬러즈 7 Colores,국가/생산지역,칠레(Chile) > 센트럴 밸리(Central Valley),주요품종,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",스타일,Chilean Cabernet Sauvignon,음용온도,16~18 ℃,...,None,None,None,None,None,None,None,None,None,None
1,생산자,GCF 그룹 GCF Group,국가/생산지역,프랑스(France) > 서던 프랑스(Southern France) > 랑그독 루씨...,주요품종,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",스타일,Languedoc-Roussillon Red,등급,Pays d'Oc IGP,...,None,None,None,None,None,None,None,None,None,None
2,생산자,가트 와인즈 Gatt Wines,국가/생산지역,호주(Australia) > 사우스 오스트레일리아(South Australia) >...,주요품종,리슬링 (Riesling) 100%,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",...,None,None,None,None,None,None,None,None,None,None
3,생산자,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,국가/생산지역,미국(U.S.A),주요품종,콩코드 (Concord),음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",...,None,None,None,None,None,None,None,None,None,None
4,생산자,빈센테 파리아 비뉴스 Vincente Faria VInhos,국가/생산지역,포르투갈(Portugal) > 도우루(Douro),주요품종,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",알코올,20~21 %,음용온도,14~16 ℃,...,None,None,None,None,None,None,None,None,None,None
5,생산자,기욤 르탕 Guillaume Letang,국가/생산지역,프랑스(France) > 서던 프랑스(Southern France) > 랑그독 루씨...,주요품종,그르나슈 (Grenache),스타일,Languedoc-Roussillon Red,숙성,프랑스 전통의 오크 통에서 3개월이상 숙성.,...,None,None,None,None,None,None,None,None,None,None
6,생산자,까브 드 루그니 Cave de Lugny,국가/생산지역,프랑스(France) > 부르고뉴(Bourgogne) > 마꼬네(Maconnais)...,주요품종,샤르도네 (Chardonnay) 100%,등급,Mâcon-Lugny AOC(AOP),알코올,13.5 %,...,None,None,None,None,None,None,None,None,None,None
7,생산자,깐띠네 포베로 Cantine Povero,국가/생산지역,이탈리아(Italy) > 피에몬테(Piemonte) > 아스티(Asti),주요품종,모스카토 (Moscato ) 100%,스타일,Italian Asti,알코올,5.5 %,...,None,None,None,None,None,None,None,None,None,None
8,생산자,운두라가 Undurraga,국가/생산지역,칠레(Chile) > 센트럴 밸리(Central Valley),주요품종,카베르네 소비뇽 (Cabernet Sauvignon),알코올,13~14 %,음용온도,16~18 ℃,...,None,None,None,None,None,None,None,None,None,None
9,생산자,까비로 Caviro sca-Forli,국가/생산지역,이탈리아(Italy) > 에밀리아 로마냐(Emilia-Romagna),주요품종,"트레비아노 (Trebbiano) , 샤르도네 (Chardonnay) , 파모소 (F...",등급,Romagna DOC(DOP),알코올,11~12 %,...,None,None,None,None,None,None,None,None,None,None


In [110]:
# 필요없는 내용들 None값으로 처리 
for i in range(5,29):
    detail_split.loc[detail_split[(i)] == "숙성",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "스타일",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "등급",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "기타정보",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "수입사",(i+1)] = 'None'

In [111]:
detail_split = detail_split.iloc[:,6:18] # 추가할 컬럼 선택 후 None값 들어갔는지 확인 

In [112]:
detail_split

,6,7,8,9,10,11,12,13,14,15,16,17
0,스타일,None,음용온도,16~18 ℃,추천음식,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",수입사,None,None,None,None,None
1,스타일,None,등급,None,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",수입사,None
2,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None,None,None
3,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None,None,None
4,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None
5,스타일,None,숙성,None,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다.",수입사,None
6,등급,None,알코올,13.5 %,음용온도,10~12 ℃,추천음식,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스...",None,None,None,None
7,스타일,None,알코올,5.5 %,음용온도,6~8 ℃,추천음식,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다.",기타정보,None,수입사,None
8,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다.",수입사,None,None,None,None,None
9,등급,None,알코올,11~12 %,음용온도,6~8 ℃,추천음식,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.,기타정보,None,*International Wine Challenge: Bronze,수입사


In [76]:
# 음용온도, 알코올, 추천음식만 가져오면 됨 

In [113]:
detail_split[detail_split[6] == '음용온도'] # 6열이 '음용온도'인 행 추출 

,6,7,8,9,10,11,12,13,14,15,16,17
2,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None,None,None
3,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None,None,None
49,음용온도,6~8 ℃,추천음식,파티에서의 바비큐 요리와 강한 소스를 감미한 해산물 요리와 완벽한 조화를 이룬다.,None,None,None,None,None,None,None,None


In [116]:
detail_split[detail_split[8] == '음용온도']

,6,7,8,9,10,11,12,13,14,15,16,17
0,스타일,None,음용온도,16~18 ℃,추천음식,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",수입사,None,None,None,None,None
4,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None
8,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다.",수입사,None,None,None,None,None
10,알코올,14~15 %,음용온도,16~19 ℃,추천음식,"진하고 풍부한 소스의 구운고기요리, 양고기 스테이크, 파스타 등과 잘 어울린다.",기타정보,None,*2011 빈티지: Robert Parker 95점,수입사,None,None
11,스타일,None,음용온도,14~16 ℃,추천음식,"흰살생선, 생선구이, 새우, 해산물, 건육, 프로슈토, 다양한 치즈, 파스타 등과 ...",None,None,None,None,None,None
12,스타일,None,음용온도,14~16 ℃,추천음식,"닭고기, 연어, 참치, 다양한 치즈, 리조또, 파스타 등과 잘 어울린다.",None,None,None,None,None,None
13,스타일,None,음용온도,16~18 ℃,추천음식,"붉은육류, 스테이크, 양갈비, 바비큐, 불고기, 파스타, 다양한 치즈, 햄버거, 한...",None,None,None,None,None,None
14,스타일,None,음용온도,14~16 ℃,추천음식,"돼지고기, 삼겹살, 베이컨, 치킨, 피자, 파스타, 다양한 치즈, 동남아요리, 아시...",None,None,None,None,None,None
15,스타일,None,음용온도,14~16 ℃,추천음식,"해산물, 조개구이, 새우, 야채, 버섯, 튀김, 아시아요리 등과 잘 어울린다.",None,None,None,None,None,None
16,등급,None,음용온도,15~17 ℃,추천음식,"양고기, 양갈비, 스테이크, 갈비, 불고기, 말린과일, 견과류 와 잘 어울린다.",수입사,None,None,None,None,None


In [96]:
detail_split[detail_split[6] == '알코올']

,6,7,8,9,10,11,12,13,14,15,16,17
1,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",수입사,None,None,None,None,None
4,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None
5,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다.",수입사,None,None,None,None,None
6,알코올,13.5 %,음용온도,10~12 ℃,추천음식,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스...",None,None,None,None,None,None
7,알코올,5.5 %,음용온도,6~8 ℃,추천음식,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다.",기타정보,None,수입사,None,None,None
8,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다.",수입사,None,None,None,None,None
9,알코올,11~12 %,음용온도,6~8 ℃,추천음식,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.,기타정보,None,*International Wine Challenge: Bronze,수입사,None,None
10,알코올,14~15 %,음용온도,16~19 ℃,추천음식,"진하고 풍부한 소스의 구운고기요리, 양고기 스테이크, 파스타 등과 잘 어울린다.",기타정보,None,*2011 빈티지: Robert Parker 95점,수입사,None,None
17,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 갈비찜, 양념갈비, 양갈비, 라자냐, 스튜, 크림파스타, 숙성 ...",수입사,None,None,None,None,None
18,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"해산물, 새우, 문어숙회, 다양한 치즈, 건육 등과 잘 어울린다.",수입사,None,None,None,None,None


In [52]:
detail_split[detail_split[8] == '알코올']

,6,7,8,9,10,11,12,13,14,15,16,17
6,등급,None,알코올,13.5 %,음용온도,10~12 ℃,추천음식,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스...",None,None,None,None
7,스타일,None,알코올,5.5 %,음용온도,6~8 ℃,추천음식,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다.",기타정보,None,수입사,None
9,등급,None,알코올,11~12 %,음용온도,6~8 ℃,추천음식,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.,기타정보,None,*International Wine Challenge: Bronze,수입사
17,스타일,None,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 갈비찜, 양념갈비, 양갈비, 라자냐, 스튜, 크림파스타, 숙성 ...",수입사,None,None,None
18,스타일,None,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"해산물, 새우, 문어숙회, 다양한 치즈, 건육 등과 잘 어울린다.",수입사,None,None,None
29,등급,None,알코올,14 %,음용온도,11~13 ℃,추천음식,"돼지고기, 바비큐, 참치, 연어, 치킨, 다양한 치즈, 하몽, 프로슈토, 살라미, ...",기타정보,None,수입사,None
30,등급,None,알코올,13~14 %,음용온도,14~16 ℃,추천음식,"가금류요리, 붉은 생선, 프와그라, 부드러운 치즈등과 잘 어울린다.",기타정보,None,"*2016 빈티지: Gilbert & Gaillard 97점, Tim Atkin 91점",*Decanter Rating - 5 stars
32,등급,None,알코올,12.5 %,음용온도,16~18 ℃,추천음식,"로스트비프, 오리고기, 닭고기, 연어, 참치, 송이버섯, 구운 아스파라거스, 콩테,...",기타정보,None,None,None
33,스타일,None,알코올,5~6 %,음용온도,6~8 ℃,추천음식,"부드러운 치즈와 커피향이 일품인 티라미수 케이크나 달콤한 과일, 특히 망고나 멜론과...",수입사,None,None,None
34,등급,None,알코올,15 %,음용온도,16~18 ℃,추천음식,"대부분의 파스타, 햄, 소시지, 치즈, 붉은 육류, 흰살 육류 등과 잘 어울린다.",기타정보,None,None,None


- 7열은 알코올, 9열 음용온도, 11열은 추천음식 

In [82]:
# 6열 음용온도 
detail_split.loc[2] = detail_split.loc[2].shift(2)
detail_split.loc[3] = detail_split.loc[3].shift(2)
detail_split.loc[49] = detail_split.loc[49].shift(2)

In [115]:
detail_split[detail_split[6] == '음용온도'].shift(2, axis=1) # 많은 데이터를 가져온 후에는 이 코드로 작업할 것! 

,6,7,8,9,10,11,12,13,14,15,16,17
2,None,None,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None
3,None,None,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None
49,None,None,음용온도,6~8 ℃,추천음식,파티에서의 바비큐 요리와 강한 소스를 감미한 해산물 요리와 완벽한 조화를 이룬다.,None,None,None,None,None,None


In [83]:
# 10열 음용온도 
detail_split.loc[6] = detail_split.loc[6].shift(-2)
detail_split.loc[7] = detail_split.loc[7].shift(-2)
detail_split.loc[9] = detail_split.loc[9].shift(-2)
detail_split.loc[17] = detail_split.loc[17].shift(-2)
detail_split.loc[18] = detail_split.loc[18].shift(-2)
detail_split.loc[29] = detail_split.loc[29].shift(-2)
detail_split.loc[30] = detail_split.loc[30].shift(-2)
detail_split.loc[32] = detail_split.loc[32].shift(-2)
detail_split.loc[33] = detail_split.loc[33].shift(-2)
detail_split.loc[34] = detail_split.loc[34].shift(-2)
detail_split.loc[40] = detail_split.loc[40].shift(-2)
detail_split.loc[42] = detail_split.loc[42].shift(-2)
detail_split.loc[43] = detail_split.loc[43].shift(-2)
detail_split.loc[44] = detail_split.loc[44].shift(-2)
detail_split.loc[45] = detail_split.loc[45].shift(-2)
detail_split.loc[47] = detail_split.loc[47].shift(-2)
detail_split.loc[48] = detail_split.loc[48].shift(-2)

In [84]:
# 12열 음용온도 
detail_split.loc[1] = detail_split.loc[1].shift(-4)
detail_split.loc[5] = detail_split.loc[5].shift(-4)
detail_split.loc[19] = detail_split.loc[19].shift(-4)
detail_split.loc[35] = detail_split.loc[35].shift(-4)
detail_split.loc[37] = detail_split.loc[37].shift(-4)
detail_split.loc[38] = detail_split.loc[38].shift(-4)
detail_split.loc[39] = detail_split.loc[39].shift(-4)
detail_split.loc[41] = detail_split.loc[41].shift(-4)
# 14열 음용온도 
detail_split.loc[25] = detail_split.loc[25].shift(-6)

In [85]:
detail_split[detail_split[8] == '음용온도']

,6,7,8,9,10,11,12,13,14,15,16,17
0,스타일,None,음용온도,16~18 ℃,추천음식,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",수입사,None,None,None,None,None
1,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",수입사,None,None,None,None,None
2,None,None,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None
3,None,None,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None
4,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None
5,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다.",수입사,None,None,None,None,None
6,알코올,13.5 %,음용온도,10~12 ℃,추천음식,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스...",None,None,None,None,None,None
7,알코올,5.5 %,음용온도,6~8 ℃,추천음식,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다.",기타정보,None,수입사,None,None,None
8,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다.",수입사,None,None,None,None,None
9,알코올,11~12 %,음용온도,6~8 ℃,추천음식,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.,기타정보,None,*International Wine Challenge: Bronze,수입사,None,None


In [87]:
df_wine["알코올"] = detail_split.get(7)
df_wine["음용온도"] = detail_split.get(9)
df_wine["추천음식"] = detail_split.get(11)

In [88]:
df_wine

,주류,중분류,상품명,생산국가,상세정보,생산자,주요품종,알코올,음용온도,추천음식
0,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,생산자\n세븐 컬러즈 7 Colores\n국가/생산지역\n칠레(Chile) > 센트...,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",None,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다."
1,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13~14 %,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
2,wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,None,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
3,wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),None,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
4,wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20~21 %,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
5,wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),14~15 %,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."
6,wine,화이트,"까브 드 루그니, 마콩 루그니 유진 블랑 샤르도네",프랑스,생산자\n까브 드 루그니 Cave de Lugny\n국가/생산지역\n프랑스(Fran...,까브 드 루그니 Cave de Lugny,샤르도네 (Chardonnay) 100%,13.5 %,10~12 ℃,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스..."
7,wine,화이트,"깐티네 포베로, 모스카토 다스티 '캄포 델 팔리오'",이탈리아,생산자\n깐띠네 포베로 Cantine Povero\n국가/생산지역\n이탈리아(Ita...,깐띠네 포베로 Cantine Povero,모스카토 (Moscato ) 100%,5.5 %,6~8 ℃,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다."
8,wine,레드,꼬모 까베르네 소비뇽,칠레,생산자\n운두라가 Undurraga\n국가/생산지역\n칠레(Chile) > 센트럴 ...,운두라가 Undurraga,카베르네 소비뇽 (Cabernet Sauvignon),13~14 %,16~18 ℃,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다."
9,wine,스파클링,노비볼레 로마냐 스푸만테 비앙코,이탈리아,생산자\n까비로 Caviro sca-Forli\n국가/생산지역\n이탈리아(Italy...,까비로 Caviro sca-Forli,"트레비아노 (Trebbiano) , 샤르도네 (Chardonnay) , 파모소 (F...",11~12 %,6~8 ℃,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.


In [90]:
# 상세정보 열 삭제 
df_wine=df_wine.drop(['상세정보'],axis=1)

In [98]:
df_wine

,주류,중분류,상품명,생산국가,생산자,주요품종,알코올,음용온도,추천음식
0,wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",None,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다."
1,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13~14 %,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
2,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,None,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
3,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),None,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
4,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20~21 %,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
5,wine,레드,"기욤 르탕, 루비 루즈",프랑스,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),14~15 %,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."
6,wine,화이트,"까브 드 루그니, 마콩 루그니 유진 블랑 샤르도네",프랑스,까브 드 루그니 Cave de Lugny,샤르도네 (Chardonnay) 100%,13.5 %,10~12 ℃,"훈제오리, 오리로스, 치킨, 훈제연어, 장어구이, 해산물, 숙성 치즈, 피자, 파스..."
7,wine,화이트,"깐티네 포베로, 모스카토 다스티 '캄포 델 팔리오'",이탈리아,깐띠네 포베로 Cantine Povero,모스카토 (Moscato ) 100%,5.5 %,6~8 ℃,"샐러드, 크림치즈, 리코타, 말린과일, 초콜릿, 스낵 등 디저트와 잘 어울린다."
8,wine,레드,꼬모 까베르네 소비뇽,칠레,운두라가 Undurraga,카베르네 소비뇽 (Cabernet Sauvignon),13~14 %,16~18 ℃,"육류, 숙성 치즈, 묵직한 소스를 곁들인 파스타 등과 잘 어울린다."
9,wine,스파클링,노비볼레 로마냐 스푸만테 비앙코,이탈리아,까비로 Caviro sca-Forli,"트레비아노 (Trebbiano) , 샤르도네 (Chardonnay) , 파모소 (F...",11~12 %,6~8 ℃,식전주 및 가벼운 생선 요리 또는 전채 요리 등과 잘 어울린다.


In [99]:
df_wine.to_csv('./wine50_fin.csv', encoding = 'utf-8-sig')